# 9 - Vlasov-Maxwell equations

Topics covered in this tutorial:

- instance of [Particels6D](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#struphy.pic.particles.Particles6D) class for PIC simulation
- phase space binning plots
- charge deposition with [AccumulatorVector](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#struphy.pic.accumulation.particles_to_grid.AccumulatorVector)
- solution of inital Poisson problem with [ImplicitDiffusion](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.ImplicitDiffusion) propagator
- particle-field coupling through the propagator [VlasovAmpere](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_coupling.html#struphy.propagators.propagators_coupling.VlasovAmpere)
- example of weak Landau damping

The equations we will solve are described in the model [VlasovAmpereOneSpecies](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_kinetic.html#struphy.models.kinetic.VlasovAmpereOneSpecies).

## Weak Landau damping

In [ ]:
# set up domain Omega
import numpy as np

from struphy.geometry.domains import Cuboid

l1 = 0.0
r1 = 12.56
l2 = 0.0
r2 = 1.0
l3 = 0.0
r3 = 1.0
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [32, 1, 1]
p = [1, 1, 1]
spl_kind = [True, True, True]
derham = Derham(Nel, p, spl_kind)

In [ ]:
# set up mass matrices
from struphy.feec.mass import WeightedMassOperators

mass_ops = WeightedMassOperators(derham, domain)

In [ ]:
# create particles object
from struphy.pic.particles import Particles6D

ppc = 10000
domain_array = derham.domain_array
nprocs = derham.domain_decomposition.nprocs
bc = ["periodic", "periodic", "periodic"]
loading_params = {"seed": None}
control_variate = True

# instantiate Particle object
particles = Particles6D(
    ppc=ppc,
    domain_decomp=(domain_array, nprocs),
    bc=bc,
    loading_params=loading_params,
    control_variate=control_variate,
    domain=domain,
)

In [ ]:
particles.draw_markers()

In [ ]:
# kinetic equilibrium
bckgr_params = {"Maxwellian3D": {"n": 1.0}}

# density perturbation for weak Landau damping
pert_params = {}
pert_params["n"] = {}
pert_params["n"]["ModesCos"] = {
    "given_in_basis": "0",
    "ls": [1],
    "amps": [0.001],
}

particles.initialize_weights(bckgr_params=bckgr_params, pert_params=pert_params)

In [ ]:
# particle binning in v1
components = [False] * 6
components[3] = True

vmin = -5.0
vmax = 5.0
n_bins = 128
bin_edges_v = np.linspace(vmin, vmax, n_bins + 1)

f_v1, df_v1 = particles.binning(components=components, bin_edges=[bin_edges_v])
print(f"{f_v1.shape = }")
print(f"{df_v1.shape = }")

In [ ]:
# plot in v1
from matplotlib import pyplot as plt

v1_bins = bin_edges_v[:-1] + (vmax - vmin) / n_bins / 2
plt.plot(v1_bins, f_v1)
plt.xlabel("vx")
plt.title("Initial Maxwellian");

In [ ]:
# particle binning in e1
components = [False] * 6
components[0] = True

emin = 0.0
emax = 1.0
bin_edges_e = np.linspace(emin, emax, n_bins + 1)

f_e1, df_e1 = particles.binning(components=components, bin_edges=[bin_edges_e])
print(f"{f_e1.shape = }")
print(f"{df_e1.shape = }")

In [ ]:
# plot in e1
e1_bins = bin_edges_e[:-1] + (emax - emin) / n_bins / 2
plt.plot(e1_bins, df_e1)
plt.xlabel("$\eta_1$")
plt.title("Initial spatial perturbation");

In [ ]:
# particle binning in e1-v1
components = [False] * 6
components[0] = True
components[3] = True

f_e1v1, df_e1v1 = particles.binning(components=components, bin_edges=[bin_edges_e, bin_edges_v])
print(f"{f_e1v1.shape = }")
print(f"{df_e1v1.shape = }")

In [ ]:
e1_bins = bin_edges_e[:-1] + (emax - emin) / n_bins / 2

plt.figure(figsize=(7, 10))

plt.subplot(2, 1, 1)
plt.pcolor(e1_bins, v1_bins, f_e1v1.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Initial Maxwellian")
plt.colorbar()

plt.subplot(2, 1, 2)
plt.pcolor(e1_bins, v1_bins, df_e1v1.T)
plt.xlabel("$\eta_1$")
plt.ylabel("$v_x$")
plt.title("Initial perturbation")
plt.colorbar();

We need to solve the Poisson equation once to get the correct initial condition for $\mathbf E$. For this we use the [ImplicitDiffusion](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.ImplicitDiffusion) propagator, see Tutorial 02. The first step is to deposit the charge to the FE grid with an [AccumulatorVector](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#struphy.pic.accumulation.particles_to_grid.AccumulatorVector) object.

In [ ]:
# accumulate charge density
from struphy.pic.accumulation.accum_kernels import charge_density_0form
from struphy.pic.accumulation.particles_to_grid import AccumulatorVector
from struphy.utils.pyccel import Pyccelkernel

# instantiate
charge_accum = AccumulatorVector(
    particles=particles,
    space_id="H1",
    kernel=Pyccelkernel(charge_density_0form),
    mass_ops=mass_ops,
    args_domain=domain.args_domain,
)

# accumulate
charge_accum(particles.vdim)

# get result
rho_vec = charge_accum.vectors[0]

In [ ]:
# use L2-projection to get density
from struphy.feec.projectors import L2Projector

l2_proj = L2Projector(space_id="H1", mass_ops=mass_ops)

rho_coeffs = l2_proj.solve(rho_vec)

In [ ]:
# fit rho coeffs into a callable field
rho = derham.create_spline_function(name="charge density", space_id="H1", coeffs=rho_coeffs)

In [ ]:
# evaluate at logical coordinates
e1 = np.linspace(0, 1, 100)
e2 = 0.5
e3 = 0.5

funval = rho(e1, e2, e3, squeeze_out=True)

In [ ]:
# plot rho in logical space
plt.plot(e1, 1e-3 * np.cos(2 * np.pi * e1), label="exact")
plt.plot(e1, funval, "--r", label="L2 projection of charge deposition")
plt.xlabel("$\eta_1$")
plt.title("Charge density for Poisson solver")
plt.legend();

In [ ]:
from struphy.propagators.propagators_fields import Poisson

# default parameters of the Propagator
opts = Poisson.options(default=True)
opts

In [ ]:
# pass simulation parameters to Propagator
Poisson.derham = derham
Poisson.domain = domain
Poisson.mass_ops = mass_ops

In [ ]:
# create solution field in Vh_0 subset H1
phi = derham.create_spline_function("my solution", "H1")

# create solution field E in Vh_1 subset H(curl)
e_field = derham.create_spline_function("electric field", "Hcurl")

In [ ]:
phi.vector

In [ ]:
e_field.vector

In [ ]:
# equation parameters
eps = 1e-12

# instantiate Propagator for the above quation, pass data structure (vector) of FemField
poisson = Poisson(phi.vector, stab_eps=eps, rho=rho.vector)

In [ ]:
# solve (call with arbitrary dt)
poisson(1.0)

In [ ]:
# compute initial E field
e_field.vector = -derham.grad.dot(phi.vector)

In [ ]:
# evalaute at logical coordinates
e1 = np.linspace(0, 1, 100)
e2 = 0.5
e3 = 0.5

e_vals = e_field(e1, e2, e3, squeeze_out=True)

In [ ]:
e_vals

In [ ]:
# plot solution
from matplotlib import pyplot as plt

plt.plot(e1, e_vals[0], label="E")
plt.xlabel("$\eta_1$")
plt.title("Initial electric field")
plt.legend();

We now create instances of the propagators [PushEta](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_markers.html#struphy.propagators.propagators_markers.PushEta) and [VlasovAmpere](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_coupling.html#struphy.propagators.propagators_coupling.VlasovAmpere), which together build the model [VlasovAmpereOneSpecies](https://struphy.pages.mpcdf.de/struphy/sections/subsections/models_kinetic.html#struphy.models.kinetic.VlasovAmpereOneSpecies).

In [ ]:
from struphy.propagators.propagators_markers import PushEta

# default parameters of Propagator
opts_eta = PushEta.options(default=True)
print(opts_eta)

In [ ]:
# pass simulation parameters to Propagator class
PushEta.domain = domain

In [ ]:
# instantiate Propagator object
prop_eta = PushEta(particles)

In [ ]:
from struphy.propagators.propagators_coupling import VlasovAmpere

# default parameters of Propagator
opts_coupling = VlasovAmpere.options(default=True)
print(opts_coupling)

In [ ]:
# pass simulation parameters to Propagator class
VlasovAmpere.domain = domain
VlasovAmpere.derham = derham
VlasovAmpere.mass_ops = mass_ops

In [ ]:
prop_coupling = VlasovAmpere(e_field.vector, particles)

In [ ]:
from time import time

import numpy as np

# diagnostics
time_vec = []
energy_E = []

# initial values
time_vec += [0.0]
energy_E += [0.5 * mass_ops.M1.dot_inner(e_field.vector, e_field.vector)]

# time stepping
Tend = 3.5
dt = 0.05
Nt = int(Tend / dt)

t = 0.0
n = 0
while t < (Tend - dt):
    t += dt
    n += 1

    t0 = time()
    # advance in time
    prop_eta(dt)
    t1 = time()
    print(f"Time for PushEta = {t1 - t0}")

    prop_coupling(dt)
    t2 = time()
    print(f"Time for VlasovAmpere = {t2 - t1}")

    print(f"Time step {n} done in {t2 - t0} sec\n")

    # diagnostics
    time_vec += [t]
    energy_E += [0.5 * mass_ops.M1.dot_inner(e_field.vector, e_field.vector)]

In [ ]:
plt.plot(time_vec, np.log(energy_E))